<a href="https://colab.research.google.com/github/MelanconR22/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,554 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:14 http://p

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-20 19:58:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-10-20 19:58:48 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
# Extract the dataset you used in Deliverable 1. 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...|2015-08-31 00:00:00|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...| 

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","review_date","product_title","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()

+--------------+-----------+----------+--------------+-------------------+--------------------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|        review_date|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+-------------------+--------------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130|2015-08-31 00:00:00|Chicago Blackhawk...|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679|2015-08-31 00:00:00|Copag Poker Size ...|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336|2015-08-31 00:00:00|Baoer 223 5.56x45...|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227|2015

In [7]:
# Create the vine_table. DataFrame
vine_df = review_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [10]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
import pyspark.sql.functions as f
vote_total_df = vine_df.filter(f.col("total_votes") >= 20)
vote_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [12]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vote_percent_df = vote_total_df.withColumn('percent_votes',f.col('helpful_votes')/f.col('total_votes')).alias('percent_votes').filter(f.col("percent_votes")>=0.5)
vote_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|               1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|               1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|               0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|           0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|0.7727272727272727|
|R18BTOE5UGN7UA|          5|  

In [13]:
# Create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid)
vine_paid_df = vote_percent_df.filter(f.col("vine") == "Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|0.8709677419354839|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|0.7619047619047619|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|0.8780487804878049|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|0.9733333333333334|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|               1.0|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|               1.0|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|              0.95|
|R1803HITBY7Y08|          4|  

In [14]:
# Repeat Step 3, retrieve all the rows where the review was not part of the Vine program (unpaid)
vine_unpaid_df = vote_percent_df.filter(f.col("vine") == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|               1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|               1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|               0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|           0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|0.7727272727272727|
|R18BTOE5UGN7UA|          5|  

In [17]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
rating_total_df = vote_percent_df.groupBy("vine").agg(
    f.count(f.col("vine")).alias("total_reviews"),
    f.count(f.when(f.col("star_rating") == 5, True)).alias("5_star_total_reviews"),
    (f.count(f.when(f.col("star_rating") == 5, True))/f.count(f.col("vine"))*100).alias("5_star_%_total")).show()

+----+-------------+--------------------+-----------------+
|vine|total_reviews|5_star_total_reviews|   5_star_%_total|
+----+-------------+--------------------+-----------------+
|   Y|          334|                 139|41.61676646706587|
|   N|        61614|               32665|53.01554841432142|
+----+-------------+--------------------+-----------------+

